In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.watchmen import Watchman
from utils.watchmen import DirectLimitWatchman, PcaLimitWatchman
from utils.watchmen import IsoForestWatchman
from utils.watchmen import LinearPredictWatchman, DeepPredictWatchman

In [4]:
SEED = 1729

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid2test_ratio': 0.3,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchhouse

In [7]:
watchhouse = {
    0: {  # GhlKasperskyDataset
        0: Watchman(),
        1: DirectLimitWatchman(),
        2: PcaLimitWatchman(),
        3: IsoForestWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
        5: DeepPredictWatchman(random_state=SEED),
    },
    1: {  # TepHarvardDataset
        0: Watchman(),
        1: DirectLimitWatchman(),
        2: PcaLimitWatchman(),
        3: IsoForestWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
        5: DeepPredictWatchman(random_state=SEED),
    },
    2: {  # TepKasperskyDataset
        0: Watchman(),
        1: DirectLimitWatchman(),
        2: PcaLimitWatchman(),
        3: IsoForestWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
        5: DeepPredictWatchman(random_state=SEED),
    },
    3: {  # SwatItrustDataset
        0: Watchman(),
        1: DirectLimitWatchman(),
        2: PcaLimitWatchman(),
        3: IsoForestWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
        5: DeepPredictWatchman(random_state=SEED),
    },
}

# Learning

## Prefit

In [8]:
for d in datasets:
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Prefit on train {d}'):
        for w in watchhouse[d]:
            watchhouse[d][w].prefit(data)

Prefit on train 0:   0%|          | 0/1 [00:00<?, ?it/s]

Prefit on train 1:   0%|          | 0/500 [00:00<?, ?it/s]

Prefit on train 2:   0%|          | 0/400 [00:00<?, ?it/s]

Prefit on train 3:   0%|          | 0/1 [00:00<?, ?it/s]

## Fit

In [9]:
for d in datasets:
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Fit on train {d}'):
        for w in watchhouse[d]:
            watchhouse[d][w].fit(data)

Fit on train 0:   0%|          | 0/1 [00:00<?, ?it/s]

Fit on train 1:   0%|          | 0/500 [00:00<?, ?it/s]

Fit on train 2:   0%|          | 0/400 [00:00<?, ?it/s]

Fit on train 3:   0%|          | 0/1 [00:00<?, ?it/s]

## Postfit

In [10]:
for d in datasets:
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Fit on train {d}'):
        for w in watchhouse[d]:
            watchhouse[d][w].postfit(data)

Fit on train 0:   0%|          | 0/1 [00:00<?, ?it/s]

Fit on train 1:   0%|          | 0/500 [00:00<?, ?it/s]

Fit on train 2:   0%|          | 0/400 [00:00<?, ?it/s]

Fit on train 3:   0%|          | 0/1 [00:00<?, ?it/s]

# Saving

## Save

In [11]:
for d in datasets:
    for w in watchhouse[d]:
        watchhouse[d][w].dump(datasets[d].__class__.__name__)

## Load

In [12]:
for d in datasets:
    for w in watchhouse[d]:
        watchhouse[d][w].load(datasets[d].__class__.__name__)